In [1]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarn

In [2]:
%%time
import os
import cv2
import numpy as np
import random

data=[]
labels = []

random.seed(40)
j = 0
listing2 = os.listdir('dataset/Accident')
for vid2 in listing2:
    vid2 = 'dataset/Accident/'+vid2
    frames = []
    cap = cv2.VideoCapture(vid2)
    ret, first_frame = cap.read()
    if first_frame is None:
        continue
    first_frame = cv2.resize(first_frame,(64,64))
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255
    i=0
    name = vid2.split("/")[1]
    while True:
        cap.set(cv2.CAP_PROP_POS_MSEC,(i * 500))
        ret, frame = cap.read()        
        print(name)
        if frame is None:
            break
        frame=cv2.resize(frame,(64,64))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray,(64,64))
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            pyr_scale = 0.5, levels = 5, winsize = 11, 
                                            iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        mask[..., 0] = angle * 180 / np.pi / 2
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
        dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
        frames.append(dense_flow)
        i=i+1
        if i > 6 and len(frames) > 6:
            break        

    input=np.array(frames)
    print (input.shape) 
    labels.append(name)
    data.append(input)

Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(

Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Acci

Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(

Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64

Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Acci

Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Acci

Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Acci

Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Accident
Accident
Accident
Accident
(7, 64, 64, 3)
Accident
Accident
Accident
Acci

In [3]:
X_tr_1= np.array(data)
print(X_tr_1.shape)

(786, 7, 64, 64, 3)


In [4]:
%%time
import os
import cv2
import numpy as np
import random


random.seed(40)
j = 0
listing2 = os.listdir('dataset/Normal')
for vid2 in listing2:
    vid2 = 'dataset/Normal/'+vid2
    frames = []
    cap = cv2.VideoCapture(vid2)
    ret, first_frame = cap.read()
    if first_frame is None:
        continue
    first_frame = cv2.resize(first_frame,(64,64))
    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255
    i=0
    while True:
        cap.set(cv2.CAP_PROP_POS_MSEC,(i * 500))
        ret, frame = cap.read()
        name = vid2.split("/")[1]
        print(name)
        if frame is None:
            break
        frame=cv2.resize(frame,(64,64))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray,(64,64))
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            pyr_scale = 0.5, levels = 5, winsize = 11, 
                                            iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        mask[..., 0] = angle * 180 / np.pi / 2
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
        dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
        frames.append(dense_flow)
        i=i+1
        if i > 6 and len(frames) > 6:
            break        

    input=np.array(frames)
    print (input.shape) 
    labels.append(name)
    data.append(input)

Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Norma

Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Norm

Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Norm

Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Norma

Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Norma

Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 6

Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
Normal
Normal
Normal
Normal
Normal
Normal
Normal
(7, 64, 64, 3)
CPU times: user 29min 29s, sys: 30.1 s, total: 30min
Wall time: 10min 29s


In [5]:
X_tr_2= np.array(data)
print(X_tr_2.shape)

(1571, 7, 64, 64, 3)


In [6]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

from keras.utils import to_categorical
# convert the labels from integers to vectors
labels = to_categorical(labels, num_classes=2)
 
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, stratify=labels, random_state=42)

In [7]:
print(data.shape, labels.shape)

(1571, 7, 64, 64, 3) (1571, 2)


In [8]:
print(labels)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [9]:
print(len(lb.classes_))

2


In [12]:
import keras
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Conv2D, Dropout
from keras import Sequential

baseModel = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=(64, 64, 3)))

model = Sequential()
model.add(
    TimeDistributed(baseModel, input_shape=(7, 64, 64, 3))
)

model.add(
    TimeDistributed(
        Flatten()
    )
)
model.add(LSTM(512, activation='relu', return_sequences=False))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(len(lb.classes_), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_3 (TimeDist (None, 7, 2, 2, 2048)     23587712  
_________________________________________________________________
time_distributed_4 (TimeDist (None, 7, 8192)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               17827840  
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 2050      
Total params: 41,942,914
Trainable params: 41,889,794
Non-trainable params: 53,120
___________________________________________________________

In [14]:
epochs = 50
print("[INFO] compiling model...")
opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
 
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(trainX, trainY, batch_size=1,
                        validation_data=(testX, testY),  epochs=epochs)

[INFO] compiling model...
[INFO] training head...
Train on 1178 samples, validate on 393 samples
Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[8192,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_2/SGD/mul_426}} = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](SGD_2/momentum/read, training_2/SGD/Variable_212/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_2/acc/Mean_1/_7337}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_15889_metrics_2/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=lb.classes_))
 
# plot the training loss and accuracy
N = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot")

In [ ]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save("accident_model")
 
# serialize the label binarizer to disk
f = open("accident_label_bin", "wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
# from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

#model = load_model("accident_model")
lb = pickle.loads(open("accident_label_bin", "rb").read())

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)
cap = cv2.VideoCapture("road.mp4")
ret, first_frame = cap.read()
first_frame = cv2.resize(first_frame, (64,64))
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)
mask[..., 1] = 255

while(True):
    frames=[]
    for i in range(0,7):
        ret, frame = cap.read()
        original = frame.copy()
        original=cv2.resize(original,(1024,768))
        if frame is None:
            break
        frame=cv2.resize(frame,(64,64))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray,(64,64))
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                            pyr_scale = 0.5, levels = 5, winsize = 11, 
                                            iterations = 5, poly_n = 5, poly_sigma = 1.1, flags = 0)
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        mask[..., 0] = angle * 180 / np.pi / 2
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
        dense_flow = cv2.addWeighted(frame, 1,rgb, 2, 0)
        x = np.array(dense_flow)
        frames.append(x)
    frames = np.array(frames)
    frames = np.expand_dims(frames,axis=0)
    preds=model.predict_proba(frames)[0]
    Q.append(preds)
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    print(label)
    text = "activity: {}".format(label)
    cv2.putText(original,text,(20,20), cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,0), 3)
    cv2.imshow('frame',original)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()